In [1]:
import pandas as pd 
import sys
sys.path.append('../../src/data_preprocessing/')

from process_doccano_output import main

class Args: 
    def __init__(self, output_folder_path, output_df_path=None, 
                 drop_bottom_level=None): 
        self.output_folder_path = output_folder_path
        self.output_df_path = output_df_path
        self.drop_bottom_level = drop_bottom_level
        
args = Args('/Users/rt853/repos/UoB/bath-persuasion-detection/data/phase_2_data/output_from_doccano/output_phase_2_partial_complete',
            None,
            True)

df = main(args)


In [18]:
labels = [
    '1-RAPPORT',
    '2-NEGOTIATE',
    '3-EMOTION',
    '4-LOGIC',
    '5-AUTHORITY',
    '6-SOCIAL',
    '7-PRESSURE',
    '8-NO-PERSUASION'
]

label2idx = {label : idx for idx, label in enumerate(labels)}


In [19]:
label2idx


{'1-RAPPORT': 0,
 '2-NEGOTIATE': 1,
 '3-EMOTION': 2,
 '4-LOGIC': 3,
 '5-AUTHORITY': 4,
 '6-SOCIAL': 5,
 '7-PRESSURE': 6,
 '8-NO-PERSUASION': 7}

In [25]:
ag = {}
for user in df.columns[1:]:
    ag[user] = []
    user_labels = df[user]
    
    for label in user_labels:
        vectors = one_hot_vector(label, label2idx)
        ag[user].append(vectors)


In [24]:
import numpy as np
def one_hot_vector(annotation, label2idx):
    empty_vec = np.zeros(len(label2idx))
    for label in annotation.split('/'): 
        if label == '8-NO':
            label = '8-NO-PERSUASION'
        empty_vec[label2idx[label]] = 1
    return empty_vec 


In [63]:
import re 

def clean_text(text):
    text = re.sub('Victim: ', '', text)
    text = re.sub('Persuader: ', '', text)
    return text 


In [65]:
from torch.nn import Softmax
import torch 

softmax = Softmax(dim=0)
idx2label = {idx : label for label, idx in label2idx.items()}
vectorised = pd.DataFrame(ag)

for idx, row in vectorised.iterrows():
    vectorised.at[idx, 'gold'] = idx2label[softmax(torch.Tensor(sum(row.values))).argmax().item()]
    
vectorised['text'] = df['text'].apply(lambda x : clean_text(x))
vectorised= vectorised[['text', 'gold']] # v  + [col for col in vectorised.columns if col != 'gold' and col != 'text']]



In [67]:
vectorised.to_csv('../../data/phase_2_data/processed/phase_2_partial_complete.csv', index=False)
